In [6]:
import pandas as pd

cols = ['sentiment', 'id', 'date', 'qurry_string', 'user', 'text']
df = pd.read_csv("./twitter_data/small_twitter_data.csv", encoding="latin1", names = cols)

data = df['text'].tolist()
print(data[0])

@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.


In [11]:
from collections import Counter
from util import *

def count_words(data):
    counter = Counter()
    for tweet in data:
        words = tweet.split()
        cleaned = map(clean_word, words)
        filtered = filter(word_not_in_stopwords, cleaned)
        counter.update(filtered)
    return counter

In [12]:
%time

counter = count_words(data)
print(counter.most_common(5))

CPU times: user 1e+03 ns, sys: 1 μs, total: 2 μs
Wall time: 2.62 μs
[('time', 45), ('night', 37), ('new', 36), ('good', 34), ('warner', 32)]


# PySpark RDD
-> Resilient Distributed Datasets (RDDs)

In [13]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('MapReduce').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf)

/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/29 09:19:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
from functools import reduce
distributed_data = sparkContext.parallelize(data, 10) # split into chunks

def mapper(text):
    words = text.split()
    cleaned = map(clean_word, words)
    filtered = filter(word_not_in_stopwords, cleaned)
    return Counter(filtered)

def reducer(counter1, counter2):
    counter1.update(counter2)
    return counter1

In [18]:
%time 
dist_data_mapped = distributed_data.map(mapper)
dist_data_reduced = dist_data_mapped.reduce(reducer)

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 3.58 μs


In [19]:
dist_data_reduced.most_common(5)

[('time', 45), ('night', 37), ('new', 36), ('good', 34), ('warner', 32)]